In [1]:
from codebase.classesmcmc import Data, MCMC
import  pystan
import argparse
import numpy as np
from codebase.file_utils import (
    save_obj,
    load_obj,
    make_folder,
    path_backslash
)
from codebase.ibis import essl, exp_and_normalise, run_mcmc
from scipy.special import expit
from tqdm.notebook import tqdm


In [7]:
log_dir = './log/20201128_174019_db2/'
expdata = load_obj('data', log_dir)

In [8]:
# log_dir = './log/mcmc_hmc_test/'
ps = load_obj('mcmc_post_samples', log_dir)

In [10]:
ps['accs']

array([1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 0.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1.,
       1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1.,
       0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0.])

In [9]:
import altair as alt
import pandas as pd
alt.data_transformers.disable_max_rows()

plot_data = pd.DataFrame(np.squeeze(ps['beta'])) 
plot_data['id'] = np.arange(len(plot_data))
plot_data = plot_data.melt(id_vars=['id'], var_name=['col'])

mcmc_chart = alt.Chart(plot_data).mark_line(
    opacity = 1,
    strokeWidth = 1,
).encode(
    alt.Y('value', title=None),
    alt.X('id:O',
          title=None
         )
).properties(width=200, height=100)

(mcmc_chart).facet(
    alt.Facet('col'),
    columns=3
)


alt.FacetChart(...)

In [5]:
b = np.squeeze(ps['beta'])
estb = np.mean(b, 0)
quantb = np.quantile(b, q=[0.025, 0.975], axis=0)
qb1 = quantb[0]
qb2 = quantb[1]
df2 = pd.DataFrame(estb, columns=['estimate'])
df2['q1'] = qb1
df2['q2'] = qb2
df2['realdata'] = expdata.raw_data['beta']
df2 = df2.reset_index()
df2

,index,estimate,q1,q2,realdata
0,0,0.927575,0.193337,1.710715,1.0
1,1,0.557583,-0.171455,1.549955,0.7
2,2,0.947167,0.119892,1.777407,0.8
3,3,0.789754,0.078845,1.773605,0.5
4,4,0.636954,-0.276851,1.405742,0.9
5,5,0.302729,-0.554961,1.099383,0.6


In [6]:
c1 = alt.Chart(df2).mark_bar(
opacity=0.6).encode(
    x='q1',
    x2='q2',
    y='index:N'
)
c2 = alt.Chart(df2).mark_point(
opacity=1,
color='blue').encode(
    x='estimate',
    y='index:N'
)
c3 = alt.Chart(df2).mark_point(
opacity=1,
color='red').encode(
    x='realdata',
    y='index:N'
)
c1+c2+c3

# red real data
# blue MCMC samples

alt.LayerChart(...)

In [7]:
def compile_model(model_num, log_dir, save=True):
    path_to_stan = './codebase/stancode/models/1factor/'

    with open('%smodel_%s.stan'%(
    path_to_stan,
    model_num
    ), 'r') as file:
        model_code = file.read()

    sm = pystan.StanModel(model_code=model_code, verbose=False)
    
    if save:
        save_obj(sm, 'sm', log_dir)
    return sm


def run_stan_model(
    data,
    compiled_model,
    num_samples, 
    num_warmup,
    num_chains,
    ):

    control={
        "metric" : "diag_e", # diag_e/dense_e
        "adapt_delta" : 0.999,
        "max_treedepth" : 25,
        "adapt_engaged" : True
        }

    fit_run = compiled_model.sampling(
        data = data,
        iter=num_samples + num_warmup,
        warmup=num_warmup,
        chains=num_chains,
#         init=0,
        control=control,
        n_jobs=1
    )

    return fit_run

def flatten_corr(a, offset = 0):
    """
    Flatten a [K, K ] correlation
    matrix to [M,] array where
    M is the number of of elements above the
    diagonal for a K by K matrix.

    Inputs
    ============
    - R : matrix to flattent out, should be
        of dimension [K,K]
    Output
    ============
    -  an array of size [M,]
    """
    return a[np.triu_indices(a.shape[0], k=offset)]
